In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib

# add directory with ribasim-repository
# sys.path.append("..\\..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
# sys.path.append("..\\..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa

import warnings

warnings.simplefilter("ignore")
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

##### SET-UP NETWORK

In [ ]:
# define network name, base dir
network_name = "tki_bommelerwaard"
base_dir = Path("..\\..\\..\\ribasim_lumping_data\\")

# directory D-HYDRO basis calculations
dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")

# directory D-HYDRO results calculations
dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")

# directory results
results_dir = Path(base_dir, network_name, "3_ribasim\\")

# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1

# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli_windows\\bin\\ribasim.exe"

Create network

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)

Load areas (discharge units: afwaterende eenheden)

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_data", "areas")
areas_file_name         = f"praktijkpeilen_bommelerwaard.shp"
areas_id_column         = "code"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column,
)
network.plot();

Read network data and extract all objects (weirs/pumps/culverts/etc) + laterals/boundaries/etc

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type="dhydro",
    set_name="winter",
    model_dir=Path(dhydro_basis_dir, "winter"),
    simulation_name="tki_vanDamvanBrakel",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=False,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
network.add_basis_network(
    source_type="dhydro",
    set_name="zomer",
    model_dir=Path(dhydro_basis_dir, "winter"),
    simulation_name="tki_vanDamvanBrakel",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=False,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
network.plot();

Export settings pumps to file and/or update settings (capacity and startlevels/stoplevels pumps)

In [ ]:
structure_specs_dir_path = Path(base_dir, network_name, "1_data", "control")
network.export_or_update_all_ribasim_structures_specs(
    structure_specs_dir_path=structure_specs_dir_path
);

#### SET-UP SIMULATION
simulation code and temporal settings

In [ ]:
simulation_code = "tki_vanDamvanBrakel"
# Definition settings changes over time
settings_time_variation = False
# in case of no variation over time. set set_name ("winter", "zomer", ...)
settings_set_name = 'winter'

Defition Lateral inflow over time
- method_laterals = 1: laterals basis network
- method_laterals = 2: laterals per discharge area > required: run function read_areas_laterals_timeseries(path_to_excel)
- method_laterals = 3: homogeneous discharge > required: define laterals_drainage_per_ha (pandas series) in liters/second/hectare

In [ ]:
network.method_laterals = 3

# definition timeseries with start-time, (number of) timesteps and the data (liters per second per hectare)
network.laterals_drainage_per_ha = pd.Series(
    index=pd.date_range("2020-01-01", periods=20, freq="D"), 
    data=[
        1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
    ]
)
network.laterals_drainage_per_ha.plot()

Definition boundaries
- method_boundaries = 1: use boundaries basis network, 
- method_boundaries = 2: use timeseries from imported csv, 
- method_boundaries = 3: use fixed boundaries

In [ ]:
network.method_boundaries = 2

boundaries_timeseries_path = Path(base_dir, network_name, "1_data", "boundaries", "boundary_timeseries.csv")
network.read_boundaries_timeseries_data(boundaries_timeseries_path, skiprows=0, sep=";")
network.boundaries_timeseries_data.plot();

Definition initial water levels (refer to simulation name and index timestep)
- method_initial_waterlevels = 1: based on one of the basis simulations (define "set_name" and index waterlevel)
- method_initial_waterlevels = 2: based on one of the simulations provided
- method_initial_waterlevels = 3: based on areas, define a column to be used

In [ ]:
network.method_initial_waterlevels = 1

network.initial_waterlevels_set_name = "winter"
network.initial_waterlevels_timestep = -6

#### ADD SPLIT NODES
use add_split_nodes function to add splitnodes. 
Select all weirs/pumps/uniweirs/orifices and/or select individual structures/objects to be included or excluded.

In [ ]:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    structures_ids_to_include=[], 
    structures_ids_to_exclude=['STW_001072B', 'STW_056492B'], 
    edge_ids_to_include=[],
);
network.plot();

Conversion of choosen split_nodes to ribasim-type. Select standard conversion for objects and individual exceptions

In [ ]:
split_node_type_conversion = {
    'weir': 'Outlet', 
    'uniweir': 'Outlet',
    'pump': 'Pump', 
    'culvert': 'Outlet', 
    'edge': 'ManningResistance',
    'orifice' : 'Outlet',
}
split_node_id_conversion = {}

##### NETWORK GENERATION

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);
network.plot();

In [ ]:
ribasim_model = network.generate_ribasim_model_complete(
    set_name=settings_set_name,
    saveat=3600,
)